# 2 Data Wrangling - for Top Video Game Review Analysis


## Contents
* 2 Data Wrangling - for Top Video Game Review Analysis
  * 2.1 Imports
  * 2.2 Load Video Game Review Dataset (vgr_data)
  * 2.3 Exploring The Video Game Review Dataset (vgr_data)
    * 2.3.1 Looking at Random game in video game review dataset
    * 2.3.2 Number Of Missing Values By Column
    * 2.3.3 Categorical Features of  Video Game Review Dataset
      * 2.3.3.1 Unique platform features



## Introduction<a id='2.2_Introduction'></a>

[Link](https://www.kaggle.com/deepcontractor/top-video-games-19952021-metacritic) Game reviews are very personal to user experiences but what interests me is if there are any similarities between game summaries, and how well they perform based on user/metacritic reviews. We will also look into how many consules are in the dataset, are there any cult classics in the top video games, are there any genres of games, can we see any trends?

So this project is also going to test if I can create a best-selling game, based on reviews/performances of best-selling games in the past.

People who love video games and grew up with certain classics have probably played one of theses games at some point.

## 2.1 Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from pandas_profiling import ProfileReport #jeff suggestion
from sklearn.preprocessing import LabelEncoder
#from library.sb_utils import save_file

There are some fundamental questions to resolve in this notebook before you move on.

* Do you think you may have the data you need to tackle the desired question?
    * Have you identified the required target value?
    * Do you have potentially useful features?
* Do you have any fundamental issues with the data?

## 2.2 Load Video Game Review Dataset (vgr_data)

In [2]:
# the supplied CSV data file is the raw_data directory
vgr_data = pd.read_csv('../raw_data/all_games.csv')

In [3]:
#What is the shape of this dataset?
vgr_data.shape

(18800, 6)

We have 18,800 rows and 6 columns.

In [4]:
#Call the info method to see a summary of the data
vgr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18800 entries, 0 to 18799
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          18800 non-null  object
 1   platform      18800 non-null  object
 2   release_date  18800 non-null  object
 3   summary       18686 non-null  object
 4   meta_score    18800 non-null  int64 
 5   user_review   18800 non-null  object
dtypes: int64(1), object(5)
memory usage: 881.4+ KB


`name` is the name of the video game.
`platform` is which console the game is played on
The other columns are when the game was released, their summary, and what score the website gave as a review, as well as the user review of that game.

I wonder if user_review and the professional reviews(meta_score)match up?

In [5]:
vgr_data.head()

,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


We can see the full name of each game, and see that the meta-score is possible based on a score out of 100 points, where as the user_review is possible out of 10 points.

We see that the release data is a date time object, and the summaries are pretty long 'str' objects. It will be interesting to see what are in some of these summaries.

## 2.3 Exploring The Video Game Review Dataset (vgr_data)

### 2.3.1 Looking at Random game in video game review dataset

In [6]:
#Since we know the shape is 18,800 entries with 6 columns, 
#lets pick a random number between 0 - 18,800
import random
x = random.randint(0, 18800)
x

13539

In [7]:
#Lets look at just one game to see what it would look like.

vgr_data.iloc[x]

name                                                         Cars
platform                                                      Wii
release_date                                    November 16, 2006
summary         Race to the finish line as you live all the fu...
meta_score                                                     65
user_review                                                   7.0
Name: 13539, dtype: object

### 2.3.2 Number Of Missing Values By Column

The video game review dataset doesn't appear to have any missing values, but lets look by counting the number of missing values in each column and sort them.

In [9]:
missing = pd.concat([vgr_data.isnull().sum(), 100 * vgr_data.isnull().mean()], axis=1)

missing.columns=['count', '%']

missing.sort_values(by= 'count')

,count,%
name,0,0.000000
platform,0,0.000000
release_date,0,0.000000
meta_score,0,0.000000
user_review,0,0.000000
summary,114,0.606383


`summary` is the only column with missing values, at 114/18800 or 0.6% a total of 114 rows have missing values.

Having a video game summary is very important because it tells us what the object of the game is and gives us a general clue as to what type of genre it is. Since it is a categorical feature, I can't just find the average for a column that had numerical values.

Can I find summaries for the missing summaries?

Now we have no missing values, but we do have categorical features.

### 2.3.3 Categorical Features of Video Game Review Dataset

Now we inspect categorical features of the healthcare dataset, of which there are many: `name` , `platform`, `release_date` and `summary` . 

In [10]:
vgr_data.select_dtypes(include='object')

,name,platform,release_date,summary,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,7.9
...,...,...,...,...,...
18795,Fast & Furious: Showdown,Xbox 360,"May 21, 2013",Fast & Furious: Showdown takes some of the fra...,1.3
18796,Drake of the 99 Dragons,Xbox,"November 3, 2003",Drake is out for revenge in a supernatural Hon...,1.7
18797,Afro Samurai 2: Revenge of Kuma Volume One,PlayStation 4,"September 22, 2015","Head out on a journey of redemption, driven by...",2.9
18798,Infestation: Survivor Stories (The War Z),PC,"October 15, 2012","(Formerly known as ""The War Z"") It has been 5 ...",1.7


Of the 6 columns of data that vgr_data has, 5 columns are categorical features.

Are there any problems with these columns? Are all of them unique?

* Are the names of the video games ALWAYS unique?
* What are the unique consoles? (platform)

#### 2.3.3.1 Unique `platform` features

In [11]:
vgr_data['platform'].value_counts()

 PC                  4864
 PlayStation 4       2056
 Xbox 360            1644
 PlayStation 2       1414
 Switch              1399
 PlayStation 3       1256
 Xbox One            1179
 Xbox                 789
 DS                   720
 Wii                  655
 PSP                  512
 GameCube             448
 Game Boy Advance     438
 3DS                  396
 PlayStation Vita     257
 PlayStation          187
 Wii U                184
 Dreamcast            125
 PlayStation 5        124
 Xbox Series X         77
 Nintendo 64           71
 Stadia                 5
Name: platform, dtype: int64

This is very interesting to find that there are only 22 different platforms to play video games on when there are in fact SO MANY MORE.


If you look at this [list](https://en.wikipedia.org/wiki/Lists_of_video_game_consoles) of video game consoles (aka `platforms`) you will see an entire range of ways to play. From hand-held consoles to dedicated video game consoles.

I don't believe that there should be such a limited range of platforms for this list. This is the top-rated video games from 1995 - 2021 and should include far more.

In [ ]:
# save the data to a new csv file
#vgr_data.to_csv('../data/all_game_data_cleaned.csv', index=False)


